In [3]:
# ==========================================
# 1. Imports & Paths
# ==========================================

from pathlib import Path
import json
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", 200)

# Reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# --- Paths ---
NB_DIR = Path.cwd()
DATA_DIR = (NB_DIR / "../data").resolve()
RES_DIR  = (NB_DIR / "../results").resolve()   # use results/ for now
RES_DIR.mkdir(parents=True, exist_ok=True)

CLEAN_CSV = DATA_DIR / "chess_games_clean.csv"

print("Data dir:", DATA_DIR)
print("Results dir:", RES_DIR)
print("Clean dataset:", CLEAN_CSV)

# --- Sklearn / classical ML ---
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, f1_score, classification_report, confusion_matrix
)
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
import joblib

# --- Deep learning (TensorFlow/Keras) ---
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("Num GPUs available:", len(tf.config.list_physical_devices('GPU')))
from tensorflow import keras
from tensorflow.keras import layers
tf.random.set_seed(RANDOM_STATE)

# --- Sequence preprocessing ---
from collections import Counter
from tensorflow.keras.preprocessing.sequence import pad_sequences

# --- Plotting ---
import matplotlib.pyplot as plt
import seaborn as sns

Data dir: E:\Github Projects\chess-outcome-prediction\data
Results dir: E:\Github Projects\chess-outcome-prediction\results
Clean dataset: E:\Github Projects\chess-outcome-prediction\data\chess_games_clean.csv


ModuleNotFoundError: No module named 'tensorflow'